In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import re
import json
import csv 

%matplotlib inline


In [2]:
!pip install emoji==0.6.0


In [3]:
import emoji


In [4]:
df=pd.read_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/Sentiment_Analysis_Dataset.csv')

In [5]:
df.head(10)

,Text,Class_camel
0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral
1,RT @___IHAVENOIDEA: حقوق المرأة في الإسلام. ht...,neutral
2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral
3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة ...,neutral
4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او...,neutral
5,RT @fatemaar7: انا كامرأة يوجعني كل حزن نساء ا...,negative
6,@Samar_K_ 😂😂😂😂😂\n اومن حقوق المرأة \n أنك تفتح...,neutral
7,@AliFCD @ShamsanM @AlyemenNor @space_dist @m2r...,negative
8,مبدائيا انا مع حقوق المرأة وضد حقوق الرجل 😁,neutral
9,RT @Dresraamohammed: السؤال بقى للناس الي شغال...,negative


In [6]:
def clean_text(text):  
    # ref: https://github.com/bakrianoo/aravec
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
    
    tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(tashkeel,"", text)
    
    # longation = re.compile(r'(.)\1+')
    # subst = r"\1\1"
    # text = re.sub(longation, subst, text)
    
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"[a-zA-Z]", '', text)
    text = re.sub(r"\d+", ' ', text)
    text = re.sub(r"\n+", ' ', text)
    text = re.sub(r"\t+", ' ', text)
    text = re.sub(r"\r+", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    text = text.strip()
    
    return text

In [7]:
# Extract each tweet's emojis - obv. it's just a brute force solution (so, it's slow) but works in ALL cases
def extract_emojis(tweets):
    emojis = []
    for tw in tweets:
        tw_emojis = []
        for word in tw:
            chars = list(word)
            for ch in chars:
                # print(ch)
                if ch in emoji.UNICODE_EMOJI:
                    tw_emojis.append(ch)
        emojis.append(' '.join(tw_emojis))
    return emojis

def contain_emoji(tweets):

  emoji___=''.join(c for c in tweets if c in emoji.UNICODE_EMOJI)  
  if len(emoji___)>0:
    return True
  return False

def convert_emojis(text):
    emoji___=''.join(c for c in text if c in emoji.UNICODE_EMOJI)
    return emoji.demojize(emoji___)

In [8]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [9]:
def extract_mention(str):
  result = re.findall("@([a-zA-Z0-9]{1,15})", str)
  return result


In [18]:
def get_url(text):
  urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
  return urls 

In [10]:
df['cleaned_text'] = df.Text.apply(clean_text)
df['cleaned_text'] = df.cleaned_text.apply(remove_urls)
df['mentions'] = df.Text.apply(extract_mention)
df["word_count"] = df["cleaned_text"].apply(lambda x: len(x.split()))

In [12]:
df["contain_emoji"] = df.Text.apply(contain_emoji)

In [16]:
df["demoji_text"] =''
df["demoji_text"] = df.Text.apply(convert_emojis)

In [20]:
df['URLS']=df.Text.apply(get_url)

In [21]:
df.head()

,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,[https://t.co/Mzf90Ta5g1]
1,RT @___IHAVENOIDEA: حقوق المرأة في الإسلام. ht...,neutral,حقوق المراه في الاسلام,[],4,False,,[https://t.co/ps3qNw1CbB]
2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"[saud, Shubratanmyeh]",15,True,:sparkles:,[https://t.co/c2NXzNCdLU]
3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة ...,neutral,حقوق المراه التي تضمنها لها وزاره العدل,[MojKsa],7,False,,[https://t.co/QUGzWwubFk]
4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او...,neutral,ولي امر الزوجه او ولي الزوجه او ولي المراه من ...,[abm112211],47,False,,[https://t.co/rkqxvm5qY7]


In [36]:
msk = np.random.rand(len(df)) < 0.7

In [37]:
train = df[msk]
test = df[~msk]

In [38]:
train.head()

,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,[https://t.co/Mzf90Ta5g1]
1,RT @___IHAVENOIDEA: حقوق المرأة في الإسلام. ht...,neutral,حقوق المراه في الاسلام,[],4,False,,[https://t.co/ps3qNw1CbB]
2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"[saud, Shubratanmyeh]",15,True,:sparkles:,[https://t.co/c2NXzNCdLU]
5,RT @fatemaar7: انا كامرأة يوجعني كل حزن نساء ا...,negative,انا كامراه يوجعني كل حزن نساء العالم سواء تعنف...,[fatemaar7],50,False,,[https://t.co/c494qBVPx4]
6,@Samar_K_ 😂😂😂😂😂\n اومن حقوق المرأة \n أنك تفتح...,neutral,اومن حقوق المراه انك تفتحلها الباب,[Samar],6,True,:face_with_tears_of_joy::face_with_tears_of_jo...,[]


In [39]:
len(train)

138720

In [40]:
test.head()

,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة ...,neutral,حقوق المراه التي تضمنها لها وزاره العدل,[MojKsa],7,False,,[https://t.co/QUGzWwubFk]
4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او...,neutral,ولي امر الزوجه او ولي الزوجه او ولي المراه من ...,[abm112211],47,False,,[https://t.co/rkqxvm5qY7]
8,مبدائيا انا مع حقوق المرأة وضد حقوق الرجل 😁,neutral,مبدائيا انا مع حقوق المراه وضد حقوق الرجل,[],8,True,:beaming_face_with_smiling_eyes:,[]
19,RT @Stranger23_: @Oh__28 @LifeRights19 عشان كذ...,negative,عشان كذا انا عندي تحفظ للي يشوف الغاء الولايه ...,"[Stranger23, Oh, LifeRights19]",49,True,:downcast_face_with_sweat:,[]
34,@Eqbal_Darandari @g11_we المفروض تتم الموافقة ...,neutral,المفروض تتم الموافقه وبلاجماع مافي ابدا توافق ...,"[Eqbal, g11]",25,False,,[]


In [41]:
len(test)

59669

In [42]:
msk = np.random.rand(len(train)) < 0.8

In [43]:
train_new = train[msk]

In [45]:
train_new.head()

,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,[https://t.co/Mzf90Ta5g1]
2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"[saud, Shubratanmyeh]",15,True,:sparkles:,[https://t.co/c2NXzNCdLU]
5,RT @fatemaar7: انا كامرأة يوجعني كل حزن نساء ا...,negative,انا كامراه يوجعني كل حزن نساء العالم سواء تعنف...,[fatemaar7],50,False,,[https://t.co/c494qBVPx4]
7,@AliFCD @ShamsanM @AlyemenNor @space_dist @m2r...,negative,غسق وسبيس وحلوه الحلوات هم اشخاص لهم هويتهم وك...,"[AliFCD, ShamsanM, AlyemenNor, space, m2rwa1, ...",32,False,,[]
9,RT @Dresraamohammed: السؤال بقى للناس الي شغال...,negative,السؤال بقي للناس الي شغاله في حقوق الانسان حقو...,[Dresraamohammed],24,True,:person_facepalming_light_skin_tone::female_si...,[]


In [46]:
len(train_new)

110922

In [44]:
valid = train[~msk]

In [47]:
valid.head()

,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
1,RT @___IHAVENOIDEA: حقوق المرأة في الإسلام. ht...,neutral,حقوق المراه في الاسلام,[],4,False,,[https://t.co/ps3qNw1CbB]
6,@Samar_K_ 😂😂😂😂😂\n اومن حقوق المرأة \n أنك تفتح...,neutral,اومن حقوق المراه انك تفتحلها الباب,[Samar],6,True,:face_with_tears_of_joy::face_with_tears_of_jo...,[]
10,نعم المرأة في الإسلام منتقصة الحقوق\n والواااا...,negative,نعم المراه في الاسلام منتقصه الحقوق والواااجبا...,[],24,False,,[]
11,RT @h747531: محاولات بائسه لاقصاء دور الام الس...,negative,محاولات بائسه لاقصاء دور الام السعوديه ودورها ...,[h747531],35,False,,[https://t.co/Cxx8oGXPD1]
15,@EslamNagyy ناصر حقوق المرأة,neutral,ناصر حقوق المراه,[EslamNagyy],3,False,,[]


In [48]:
len(valid)

27798

In [49]:
train_new.to_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/train.csv')

In [50]:
valid.to_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/val.csv')

In [51]:
test.to_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/test.csv')